# Test of OFDM Class

In [1]:
from OFDM import *

Define the OFDM Modulation scheme using:

CamG(subcarriers, cyclic prefix, modulation method, pilot=0)

In [2]:
test = CamG(64, 16, "QPSK")

Generate some random bits the length of the OFDM symbol

In [3]:
bits = np.random.binomial(n=1, p=0.5, size=(test.bits_per_symbol, ))
print ("Bit count: ", len(bits))

Bit count:  128


## Transmitter
Convert them to parallel stream

(this only currently works when the length of bits = number of data carriers x bits per constellation symbol i.e. we only transmit one OFDM symbol)

In [4]:
bits_SP = test.SP(bits)
print(bits_SP.shape)

(64, 2)


Map them to symbols using the determined modulation scheme

In [5]:
QPSK = test.map(bits_SP)
print ("First 5 symbols and bits:")
print (bits_SP[:5,:])
print (QPSK[:5])

First 5 symbols and bits:
[[1 1]
 [0 1]
 [1 0]
 [0 1]
 [1 1]]
[-0.70710678-0.70710678j  0.70710678-0.70710678j -0.70710678+0.70710678j
  0.70710678-0.70710678j -0.70710678-0.70710678j]


Map onto carriers with pilot symbols

In [6]:
OFDM_data = test.OFDM_symbol(QPSK)
print("Number of OFDM carrieres in frequency domain:", len(OFDM_data))

Number of OFDM carrieres in frequency domain: 64


Compute IDFT of signal data and add CP

In [7]:
OFDM_time = IFFT(OFDM_data)
OFDM_with_cp = test.add_cp(OFDM_time)
print ("Number of OFDM samples in time domain with CP: ", len(OFDM_with_cp))

Number of OFDM samples in time domain with CP:  80


## Receiver
Remove the CP and compute the FFT

In [8]:
OFDM_rx = OFDM_with_cp
OFDM_rx = test.remove_cp(OFDM_rx)
OFDM_demod = FFT(OFDM_rx)

Estimate the channel coefficients and equalise the demodulated OFDM symbol

We'll just assume the channel is ideal for now - channel estimation not yet implemented

In [9]:
h_est = np.ones(len(test.all_carriers))
OFDM_demod = equalise(OFDM_demod,h_est)

Extract the data carriers and disguard the pilot values

In [10]:
QPSK_est = test.get_data(OFDM_demod)

Demap the constellation symbols to bits using minimum distance rule

In [11]:
bits_PS, descision_symbols = test.demap(QPSK_est)

Convert parallel streams to serial data stream

In [12]:
decoded_bits = test.PS(bits_PS)

Check to see if we received what we transmitted

In [17]:
np.testing.assert_array_equal(decoded_bits,bits)